## The Oracle

In quantum computing, an oracle is a device used to encode information about a function without explicitly revealing its explicit rule, it is also known as black box. It plays a crucial role in many quantum algorithms, such as Grover's search algorithm and the Deutsch-Jozsa algorithm. The oracle can be thought of as a tool that, when given a specific input, produces an output according to an unknown function $f(x)$. 

Great! But how do we represent and construct an oracle in a quantum circuit? In general, an oracle is represented by a unitary operator $U_f$. This operator acts on a quantum state to evaluate a function $f(x)$. For example, in the context of Grover's [[3](#grover_paper), [4](#childs_notes)] and Deutsch-Jozsa algorithm [[1](#dj_paper), [2](#dj_tutorial), [4](#childs_notes)], the oracle $U_f$ takes the action $U_f|x\rangle |y\rangle = |x\rangle |y\oplus f(x)\rangle$. Both of these algorithms provide speedups in the number of steps required for its execution, when compared to its classical counterparts. 

![Oracle_fig](https://docs.classiq.io/resources/Oracle_fig.png)

However, in general, the construction of an oracle is not a trivial task. Implementing a quantum oracle requires encoding the desired function into a quantum circuit. This often involves designing a series of quantum gates that represent the function's logic. While the specifics can vary, the general approach is to create a circuit where the function $f(x)$ influences the qubits in a way that reflects its output.

# Classiq Concepts

In this tutorial, we will be using some Classiq elements that you can take a look:

* [Arithmetic operations](https://docs.classiq.io/latest/explore/tutorials/technology_demonstrations/arithmetic_expressions/arithmetic_expressions/)

You can also take a look at our [Deutsch-Jozsa Tutorial](https://docs.classiq.io/latest/explore/algorithms/deutsch_jozsa/deutsch_jozsa/) and [Grover's Algorithm Tutorial](https://docs.classiq.io/latest/explore/built_in_apps/grover/grover/) if you want to see an oracle being applied in a quantum algorithm.

One of the main challenges in using quantum oracles is constructing efficient and scalable representations for complex functions. As quantum computing technology advances, finding optimal methods for oracle construction and integration into quantum algorithms remains an active area of research. For instance, let's examine the oracle for the following function, which can be used in the context of the [Deutsch-Jozsa algorithm](https://docs.classiq.io/latest/explore/algorithms/deutsch_jozsa/deutsch_jozsa/), as a balanced function:

![Oracle_table](https://docs.classiq.io/resources/Oracle_table.png)

<details><summary>How do we build the oracle for the function displayed in the table?</summary>

The function $f(x)$ assumes its value as $1$ only when the integer value of $| x \rangle$ is even. Therefore, everytime the last qubit of $| x \rangle$ is $1$, $f(x)=1$ and our oracle should simply be a CNOT over the last qubit of $| x \rangle$.

</details>

We can see that the function $f(x)$ follows a rule: Everytime the integer value of the qubit $| x \rangle$ is divisible by $2$, $f$ is $1$. In other words, if we were to build an oracle for this function, it should flip the $y$ bit every time $x = 0$ mod $2$. Even though, in this case, we can construct such oracle in a clever way, it is not always an easy task to build it.

If we have an arithmetic expression for the rule of $f(x)$, it is possible to construct this oracle with a fre lines of code in Classiq, letting the building blocks of the quantum circuit being automatically done by the synthesizer, and even being able to optimize it in the depth of the circuit (number of operations) or in the width (number of qubits):

In [1]:
from classiq import *

@qfunc
def oracle_function(res:QNum, x:QNum):
    res ^= x %2 == 1

Thats it! Our oracle is constructed. In Classiq's IDE, it would look like this:

![Oracle_gif](https://docs.classiq.io/resources/Oracle_gif.gif)

Now we are able to append it as a subroutine in a quantum circuit. For example, we can apply it in the Deutsch-Jozsa algorithm:

In [4]:
@qfunc
def main(y: Output[QNum], x: Output[QNum]):
    
    allocate(3,x)
    allocate(1,y)

    X(target=y)
    apply_to_all(H,x)
    oracle_function(y,x)
    apply_to_all(H,x)

qmodel = create_model(main)
qprogram = synthesize(qmodel)
show(qprogram)

Opening: https://platform.classiq.io/circuit/8e59c536-8a52-41d5-a098-de54a1d0031c?version=0.42.2


In the IDE, the Deutsch-Jozsa algorithm using this oracle would look like this:

![DJ_gif](https://docs.classiq.io/resources/DJ-circuit.gif)

# References

<a id='dj_paper'>[1]</a>: [Rapid solution of problems by quantum computation (David Deutsch and Richard Jozsa)](https://doi.org/10.1098/rspa.1992.0167)

<a id='dj_tutorial'>[2]</a>: [Deutsch-Jozsa Algorithm Tutorial (Classiq's documentation)](https://docs.classiq.io/latest/explore/algorithms/deutsch_jozsa/deutsch_jozsa/)

<a id='grover_paper'>[3]</a>: [A fast quantum mechanical algorithm for database search (Lov K. Grover)](https://doi.org/10.1145/237814.237866)